In [1]:
# gbm이란 틀린 부분에 가중치를 더하면서 진행하는 알고리즘

# # 안건들여도 될것 같은거
#     boosting_type: str = 'gbdt',
#     learning_rate: float = 0.1, - 건들면 좋을 것 같긴한데 학습시간에 많은 영향이 있을 것으로 예상되어 마지막에
#     objective: Union[str, Callable, NoneType] = None,
#     class_weight: Union[Dict, str, NoneType] = None,
#     random_state: Union[int, numpy.random.mtrand.RandomState, NoneType] = None,
#     subsample: float = 1.0, - # 훈련 인스턴스의 하위 샘플 비율입니다
#     subsample_freq: int = 0, - # 서브샘플의 빈도, <=0은 활성화가 없음을 의미
#     n_jobs: int = -1, - # 병렬 스레드 수, 병렬처리 여부, -1이면 모든 코어 사용
#     silent: Union[bool, str] = 'warn', - # 부스팅을 실행하는 동안 메시지를 인쇄할지 여부
#     importance_type: str = 'split',

# # 100까지 올렸을 때 정확도가 올라가는 것으로 보임, 딱히 과적합이 양상이 보이지 않음
# # 120부터 4.2대로 나오기 시작했지만 150번째에서 4.19가 찍힘
#     num_leaves: int = 31, - 전체 Tree의 leave 수 이고, 디폴트값은 31
# 	- 개별 트리가 가질 수 있는 최대 리프의 개수를 지정, lightgbm복잡도를 제어하는 주 파라미터
# 	- 수가 커질 수록 모델의 깊이가 깊어지므로 모델 정확도가 높아지지만 과적합 발생 확률 또한 높아짐
# # 맥스딥스로는 큰 효과를 못봄, -1, 0, 11, 12, 에서 5.06, 5.07정도의 값이 나옴 15에서 5.02가 나오긴 함
#     max_depth: int = -1,  - tree의 최대 깊이, 모델 과적합을 다룰때 사용, 과적합으로 느끼면 값을 줄여야함
# # 적합하도록 부스트된 나무의 수
#     n_estimators: int = 100, - # 트리의 갯수 높을 수록 정확도가 높아지나 시간이 오래 걸림



# # 과적합 개선에 사용되는 파라미터
#     reg_alpha: float = 0.0, - # 가중치에 대한 L1 정규화 항
#     reg_lambda: float = 0.0, - # 가중치에 대한 L2 정규화 항



# ------------------------------------------------------------------------------------------------------------------
# # bin구성을 위한 샘플 수
#     subsample_for_bin: int = 200000,

# # 트리의 리프 노드에서 추가 파티션을 만드는 데 필요한 최소 손실 감소
#     min_split_gain: float = 0.0,

# # 자식(리프)에 필요한 인스턴스 가중치(헤시안)의 최소 합계
#     min_child_weight: float = 0.001,

# # 자식(리프)에 필요한 최소 데이터
#     min_child_samples: int = 20,

# # 각 트리를 구성할 때 열의 하위 샘플 비율
#     colsample_bytree: float = 1.0,




In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

In [4]:
# 데이터 로드

train = pd.read_parquet('C:/Users/knzss/Documents/ds_study/project/ds5-ml-repo-1/data/train_fin.parquet')
train.head()

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target,weekend,month
0,TRAIN_0000000,20220623,1,17,1,106,지방도1112호선,0,0,60.0,...,33.427747,126.662612,0,제3교래교,33.427749,126.662335,0,52.0,0,6
1,TRAIN_0000001,20220728,1,21,2,103,일반국도11호선,0,0,60.0,...,33.500730,126.529107,1,KAL사거리,33.504811,126.526240,0,30.0,0,7
2,TRAIN_0000002,20211010,4,7,2,103,일반국도16호선,0,0,80.0,...,33.279145,126.368598,0,상창육교,33.280072,126.362147,0,61.0,1,10
3,TRAIN_0000003,20220311,0,13,2,107,태평로,0,0,50.0,...,33.246081,126.567204,0,서현주택,33.245565,126.566228,0,20.0,0,3
4,TRAIN_0000004,20211005,6,8,2,103,일반국도12호선,0,0,80.0,...,33.462214,126.326551,0,애월입구,33.462677,126.330152,0,38.0,0,10


In [5]:
from sklearn.utils import shuffle

tr = shuffle(train.sort_values('base_date')[train['base_date'] < 20220701])
te = shuffle(train.sort_values('base_date')[train['base_date'] > 20220631])

y_train = tr['target']
X_train = tr.drop(['day_of_week', 'multi_linked', 'connect_code', 'height_restricted', 'id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

y_test = te['target']
X_test = te.drop(['day_of_week', 'multi_linked', 'connect_code', 'height_restricted', 'id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)


print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

C:\Users\knzss\AppData\Local\Temp\ipykernel_8796\489013793.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tr = shuffle(train.sort_values('base_date')[train['base_date'] < 20220701])
C:\Users\knzss\AppData\Local\Temp\ipykernel_8796\489013793.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  te = shuffle(train.sort_values('base_date')[train['base_date'] > 20220631])


(4426522, 14)
(4426522,)
(274695, 14)


In [56]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV



LR = lgb.LGBMRegressor()

### parameter 들을 dictionary 형태로 설정
parameters = {'max_depth':[15],
              'num_leaves':[187, 188],
              'learning_rate' : [0.5, 0.6]}

# param_grid의 하이퍼 파라미터들을 3개의 train, test set fold 로 나누어서 테스트 수행 설정.  
### refit=True 가 default 임. True이면 가장 좋은 파라미터 설정으로 재 학습 시킴.  
grid_lr = GridSearchCV(LR, param_grid=parameters, cv=3, refit=True)

# 붓꽃 Train 데이터로 param_grid의 하이퍼 파라미터들을 순차적으로 학습/평가 .
grid_lr.fit(X_train, y_train)

# GridSearchCV 결과 추출하여 DataFrame으로 변환
scores_df = pd.DataFrame(grid_lr.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 
           'split0_test_score', 'split1_test_score', 'split2_test_score']]


,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'learning_rate': 0.5, 'max_depth': 15, 'num_l...",0.913571,3,0.913016,0.912929,0.914769
1,"{'learning_rate': 0.5, 'max_depth': 15, 'num_l...",0.913552,4,0.912757,0.912828,0.915072
2,"{'learning_rate': 0.6, 'max_depth': 15, 'num_l...",0.913855,1,0.913297,0.913152,0.915114
3,"{'learning_rate': 0.6, 'max_depth': 15, 'num_l...",0.913805,2,0.913251,0.913030,0.915134


In [99]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

LR = lgb.LGBMRegressor()

parameters = {'max_depth':[15, 16],
              'num_leaves':[187, 188],
              'learning_rate' : [0.5, 0.6],
              'n_estimators' : [293, 294]
             }

grid_lr = GridSearchCV(LR, param_grid=parameters, cv=3, refit=True)

grid_lr.fit(X_train, y_train)

scores_df = pd.DataFrame(grid_lr.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 
           'split0_test_score', 'split1_test_score', 'split2_test_score']]


,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'learning_rate': 0.5, 'max_depth': 15, 'n_est...",0.916434,8,0.915852,0.915851,0.917597
1,"{'learning_rate': 0.5, 'max_depth': 15, 'n_est...",0.916441,6,0.915812,0.915736,0.917776
2,"{'learning_rate': 0.5, 'max_depth': 15, 'n_est...",0.916439,7,0.915857,0.915861,0.917600
3,"{'learning_rate': 0.5, 'max_depth': 15, 'n_est...",0.916447,5,0.915817,0.915742,0.917781
4,"{'learning_rate': 0.5, 'max_depth': 16, 'n_est...",0.916474,4,0.915746,0.915862,0.917815
5,"{'learning_rate': 0.5, 'max_depth': 16, 'n_est...",0.916591,2,0.915974,0.915931,0.917869
6,"{'learning_rate': 0.5, 'max_depth': 16, 'n_est...",0.916478,3,0.915746,0.915869,0.917818
7,"{'learning_rate': 0.5, 'max_depth': 16, 'n_est...",0.916596,1,0.915977,0.915934,0.917877
8,"{'learning_rate': 0.6, 'max_depth': 15, 'n_est...",0.916294,10,0.915723,0.915546,0.917614
9,"{'learning_rate': 0.6, 'max_depth': 15, 'n_est...",0.916258,15,0.915666,0.915573,0.917534


In [102]:
grid_lr.cv_results_['params'][7]

{'learning_rate': 0.5, 'max_depth': 16, 'n_estimators': 294, 'num_leaves': 188}

In [61]:
f_list = [i for i in range(15, 50)]
mae_list = []
min_num = 10
min_stack = 0
stop_num = 3

for i in f_list:

    LR = lgb.LGBMRegressor(random_state = 42,
                           max_depth = i,
                           num_leaves = 187,
                           learning_rate = 0.6,
                           n_estimators = 177,
    #                        reg_lambda = 0.01
                          ).fit(X_train, y_train)

    pred = LR.predict(X_test)
    mae = mean_absolute_error(pred, y_test)
    print(mae)

    if min_num > mae:
        min_stack = 0
        mae_list.append((i, mae))
        min_num = mae
        print('max_depth : ', i, ', mae : ', mae)
    elif min_stack != stop_num:
        min_stack += 1
        mae_list.append((i, mae))
        min_num = mae
        print('max_depth : ', i, ', mae : ', mae)
    elif min_stack == stop_num:
        break
#         pass

3.6203461223850097
max_depth :  15 , mae :  3.6203461223850097
3.61689002167885
max_depth :  16 , mae :  3.61689002167885
3.617534303895066
max_depth :  17 , mae :  3.617534303895066
3.6128050674436416
max_depth :  18 , mae :  3.6128050674436416
3.6188660343306136
max_depth :  19 , mae :  3.6188660343306136
3.6082254470161836
max_depth :  20 , mae :  3.6082254470161836
3.6205621752937276
max_depth :  21 , mae :  3.6205621752937276
3.609805600108715
max_depth :  22 , mae :  3.609805600108715
3.6341644694917266
max_depth :  23 , mae :  3.6341644694917266
3.6088862549258134
max_depth :  24 , mae :  3.6088862549258134
3.6236360059351074
max_depth :  25 , mae :  3.6236360059351074
3.621990894306523
max_depth :  26 , mae :  3.621990894306523
3.612335881958639
max_depth :  27 , mae :  3.612335881958639
3.616587180685243
max_depth :  28 , mae :  3.616587180685243
3.6191680076626516
max_depth :  29 , mae :  3.6191680076626516
3.605686624984051
max_depth :  30 , mae :  3.605686624984051
3.605686

In [102]:
# grid_dtree.cv_results_
LR = lgb.LGBMRegressor(random_state = 42,
                           max_depth = 19,
                           num_leaves = 187,
                           learning_rate = 0.3,
                           n_estimators = 1000,
                          ).fit(X_train, y_train)

pred = LR.predict(X_test)
mae = mean_absolute_error(pred, y_test)
print(mae)

4.4864842041593835


In [65]:
importances = pd.Series(LR.feature_importances_, X_train.columns)
importances

base_hour                8889
lane_count               1110
road_rating               934
maximum_speed_limit       974
weight_restricted         399
road_type                 177
start_latitude           4066
start_longitude          3227
start_turn_restricted     157
end_latitude             3225
end_longitude            2488
end_turn_restricted       109
weekend                  1578
month                    5589
dtype: int32

In [ ]:
from sklearn.metrics import mean_absolute_error


mae_list = []
min_num = 10
min_stack = 0
stop_num = 3

# test, max_depth
# 맥스딥스로는 큰 효과를 못봄, -1, 0, 11, 12, 에서 5.06, 5.07정도의 값이 나옴 15에서 5.02가 나오긴 함
for i in range(-1, 20):
    LR = lgb.LGBMRegressor(random_state = 42,
                           max_depth = i
                          ).fit(X_train, y_train)

    pred = LR.predict(X_test)
    mae = mean_absolute_error(pred, y_test)
    
    if min_num > mae:
        min_stack = 0
        mae_list.append((i, mae))
        min_num = mae
        print('max_depth : ', i, ', mae : ', mae)
    elif min_stack != stop_num:
        min_stack += 1
        mae_list.append((i, mae))
        min_num = mae
        print('max_depth : ', i, ', mae : ', mae)
    elif min_stack == stop_num:
        break
            

In [6]:
# test, num_leaves
# 100까지 올렸을 때 정확도가 올라가는 것으로 보임, 딱히 과적합이 양상이 보이지 않음
# 120부터 4.2대로 나오기 시작했지만 150번째에서 4.19가 찍힘
default = 130
max_num_t = 150
min_num = 10

for i in range(default, max_num_t + 1):
    LR = lgb.LGBMRegressor(random_state = 42,
                           num_leaves  = i
                          ).fit(X_train, y_train)

    pred = LR.predict(X_test)
    mae = mean_absolute_error(pred, y_test)
    
    if min_num > mae:
        min_stack = 0
        min_num = mae
        print('num_leaves  : ', i, ', mae : ', mae)
    elif min_stack != stop_num:
        min_stack += 1
        min_num = mae
        print('num_leaves  : ', i, ', mae : ', mae)
    elif min_stack == stop_num:
        break

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import mean_absolute_error

default = 293
max_num_t = 320

min_num = 10
min_stack = 0
stop_num = 3

# for i in range(default, max_num_t + 1):
LR = lgb.LGBMRegressor(random_state = 42,
                       max_depth = 19, # 원핫 - 20
                       num_leaves = 187, # 200
                       n_estimators = 1000,
                       learning_rate = 0.3,

#                            min_split_gain = (i / 100), # 0.0 mae값이 올라버림, 값이 오름에 따라
#                            min_child_weight = (i / 1000), # 영향 미비
#                            min_child_samples = i, # 20 미비
#                            colsample_bytree = (i / 10), # 1.0이 젤 좋음
#                            reg_lambda = (i / 100),
                      ).fit(X_train, y_train)

pred = LR.predict(X_test)
mae = mean_absolute_error(pred, y_test)
print(mae)
#     if min_num > mae:
#         min_stack = 0
#         min_num = mae
#         print('파라미터 값  : ', i, ', mae : ', mae)
#     elif min_stack != stop_num:
#         min_stack += 1
#         min_num = mae
#         print('파라미터 값  : ', i, ', mae : ', mae)
#     elif min_stack == stop_num:
#         break